In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from ast import literal_eval
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import ParameterGrid
from keras.utils import to_categorical
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import os
import openpyxl
import random
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
import json
import pandas as pd
import random


# Spilt test

In [ ]:
# Legitimate
def load_legit_data(json_path):
    with open(json_path, 'r') as f:
        raw_data = json.load(f)
    
    data_type = None
    if isinstance(raw_data, list):
        if all(isinstance(x, list) for x in raw_data):  
            data_type = "list_of_lists"
        elif all(isinstance(x, dict) for x in raw_data):  
            data_type = "list_of_dicts"
    elif isinstance(raw_data, dict) and "embeddings" in raw_data:  
        data_type = "embeddings_dict"
    
    if not data_type:
        raise ValueError("Unrecognized Legal Sample Format")
    
    return raw_data, data_type

def split_and_save_legit(orig_data, data_type, split_size, output_path_1, output_path_2):
    if data_type == "list_of_lists":
        df = pd.DataFrame(orig_data)
    elif data_type == "list_of_dicts":
        df = pd.DataFrame([x["embedding"] for x in orig_data])
    elif data_type == "embeddings_dict":
        df = pd.DataFrame(orig_data["embeddings"])
    
    if split_size > len(df):
        raise ValueError(f"number of segments{split_size}exceeding the dataset size{len(df)}")
    
    sampled = df.sample(n=split_size, random_state=42)
    remaining = df.drop(sampled.index)
    
    print(f"Segmentation Statistics")
    print(f"├─ Total Number Of Original Samples: {len(df)}")
    print(f"├─ Number Of Samples: {len(sampled)}")
    print(f"└─ Number Of Samples Remaining: {len(remaining)}\n")
    
    def _convert_back(data_df, orig_type):
        if orig_type == "list_of_lists":
            return data_df.values.tolist()
        elif orig_type == "list_of_dicts":
            return [{"embedding": row.tolist()} for _, row in data_df.iterrows()]
        elif orig_type == "embeddings_dict":
            return {"embeddings": data_df.values.tolist()}
    
    def _save_data(data, path, orig_type):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'w') as f:
            json.dump(data, f, indent=2)
    
    _save_data(_convert_back(sampled, data_type), output_path_1, data_type)
    _save_data(_convert_back(remaining, data_type), output_path_2, data_type)

if __name__ == "__main__":
    input_path = "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/legiti_embeddings_js.json"          # the path of the original legal sample
    output_sampled = "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/legiti_test_js.json"     # the sample path was drawn
    output_remaining = "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/legiti_train_js.json" # remaining sample paths
    split_num = 400    # spilt number

    try:
        orig_data, data_format = load_legit_data(input_path)
        
        split_and_save_legit(
            orig_data = orig_data,
            data_type = data_format,
            split_size = split_num,
            output_path_1 = output_sampled,
            output_path_2 = output_remaining
        )
        
        print(f"Successfully Split And Save：\nSamples Were Drawn -> {output_sampled}\nRemaining Samples -> {output_remaining}")
    
    except Exception as e:
        print(f"处理失败：{str(e)}")

In [ ]:
# Malicious Packages

CONFIG = {
    "input_path": "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/malware_clusters_results_js.json",
    "output_sampled": "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/mal_test_js.json",
    "output_remaining": "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/mal_train_js.json",
    "total_samples": 400,
    "allow_repeats": False
}

def main():
    input_file = CONFIG["input_path"]
    
    try:
        print(f"Reading Input File: {input_file}")
        with open(input_file, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)
    except Exception as e:
        exit(f"Failed to read file: {str(e)}")

    print(f"Cluster sampling started. Number of requested samples: {CONFIG['total_samples']}")
    try:
        # Perform cluster sampling and preserve the original structure
        sampled_data, remaining_data, stats = cluster_sampling(
            raw_data, 
            CONFIG["total_samples"],
            CONFIG["allow_repeats"]
        )
        
        save_cluster_data(sampled_data, CONFIG["output_sampled"])
        save_cluster_data(remaining_data, CONFIG["output_remaining"])
        
    except Exception as e:
        exit(f"Error: {str(e)}")

    # Print detailed sampling statistics
    print_statistics(stats)

def print_statistics(stats):
    total_sampled = sum(info['sampled'] for info in stats.values())
    total_remaining = sum(info['remaining'] for info in stats.values())
    
    print("\n=== Cluster Sampling Statistics ===")
    print(f"Total Sample Requested: {CONFIG['total_samples']}")
    print(f"Actual Number of Samples Drawn: {total_sampled}")
    print(f"Total Remaining Samples: {total_remaining}\n")
    print("Per-Cluster Details:")
    print(f"{'Cluster ID':<10} {'Allocated':<8} {'Original':<8} {'Sampled':<8} {'Remaining':<8} {'Status':<12}")
    print("-" * 65)
    for cluster_id, info in stats.items():
        status = "With Repeats" if info['repeats_used'] else "Normal"
        print(f"{cluster_id:<10} {info['allocated']:<8} {info['original']:<8} "
              f"{info['sampled']:<8} {info['remaining']:<8} {status:<12}")


def save_cluster_data(data, path):
    """Save cluster data (preserve original format)"""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    # Auto-detect original format
    if isinstance(data, list):
        output = []
        for cluster in data:
            output.append({
                cluster['id']: {
                    "size": cluster['size'],
                    "samples": cluster['samples']
                }
            })
    else:
        output = {
            cluster['id']: {
                "size": cluster['size'],
                "samples": cluster['samples']
            } for cluster in data.values()
        }
    
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=2, ensure_ascii=False)

def cluster_sampling(raw_data, total_samples, allow_repeats):
    """Improved cluster sampling logic ensuring each cluster has at least one sample,
       and remaining samples are allocated proportionally."""
    
    # Normalize structure
    if isinstance(raw_data, list):
        processed_data = {}
        for item in raw_data:
            processed_data.update(item)
    else:
        processed_data = raw_data

    # Remove invalid clusters (size <= 0)
    valid_clusters = {
        k: v for k, v in processed_data.items() 
        if isinstance(v.get("size", 0), (int, float)) and v["size"] > 0
    }
    if not valid_clusters:
        raise ValueError("No valid clusters found (all have size <= 0)")

    # Initialize stats dicts
    stats = {}
    sampled_clusters = {}
    remaining_clusters = {}

    # Total cluster count
    num_clusters = len(valid_clusters)
    if total_samples < num_clusters:
        raise ValueError(f"Not enough total samples {total_samples} to assign at least one per cluster.")

    # Pre-assign 1 sample per cluster
    base_alloc = {cluster_id: 1 for cluster_id in valid_clusters}
    remaining_samples = total_samples - num_clusters

    # Total size for proportional allocation
    total_size = sum(details["size"] for details in valid_clusters.values())

    # Proportional allocation of remaining samples
    allocations = base_alloc.copy()
    for cluster_id, details in valid_clusters.items():
        if total_size > 0:
            alloc = round((details["size"] / total_size) * remaining_samples)
        else:
            alloc = 0  # Prevent division by zero
        allocations[cluster_id] += alloc

    # Fix potential rounding discrepancy
    allocated_sum = sum(allocations.values())
    if allocated_sum < total_samples:
        # Add leftover to largest cluster
        largest_cluster_id = max(valid_clusters.items(), key=lambda x: x[1]["size"])[0]
        allocations[largest_cluster_id] += (total_samples - allocated_sum)

    # Sample from clusters
    for cluster_id, details in valid_clusters.items():
        original_samples = details["samples"]
        need = allocations.get(cluster_id, 0)

        # Init cluster stats
        cluster_stat = {
            "original": len(original_samples),
            "allocated": need,
            "sampled": 0,
            "remaining": len(original_samples),
            "repeats_used": False
        }

        if need > 0:
            if len(original_samples) < need:
                if not allow_repeats:
                    print(f"Warning: Cluster {cluster_id} has insufficient samples (requested {need}, available {len(original_samples)}). Adjusted.")
                    need = len(original_samples)
                    cluster_stat["allocated"] = need
                else:
                    print(f"Notice: Cluster {cluster_id} using repeated sampling (requested {need}, available {len(original_samples)}).")
                    cluster_stat["repeats_used"] = True

            if allow_repeats:
                selected = random.choices(original_samples, k=need)
                remaining_samples_list = original_samples
            else:
                selected = random.sample(original_samples, min(need, len(original_samples)))
                remaining_samples_list = [s for s in original_samples if s not in selected]

            cluster_stat.update({
                "sampled": len(selected),
                "remaining": len(remaining_samples_list)
            })

            sampled_clusters[cluster_id] = {
                "id": cluster_id,
                "size": len(selected),
                "samples": selected
            }
            remaining_clusters[cluster_id] = {
                "id": cluster_id,
                "size": len(remaining_samples_list),
                "samples": remaining_samples_list
            }
        else:
            remaining_clusters[cluster_id] = {
                "id": cluster_id,
                "size": details["size"],
                "samples": details["samples"]
            }

        stats[cluster_id] = cluster_stat

    # Handle invalid clusters
    for cluster_id in set(processed_data.keys()) - set(valid_clusters.keys()):
        stats[cluster_id] = {
            "original": len(processed_data[cluster_id]["samples"]),
            "allocated": 0,
            "sampled": 0,
            "remaining": len(processed_data[cluster_id]["samples"]),
            "repeats_used": False
        }
        remaining_clusters[cluster_id] = processed_data[cluster_id]

    output_format = type(raw_data)
    return (
        output_format([{k:v} for k,v in sampled_clusters.items()]) if isinstance(raw_data, list) else sampled_clusters,
        output_format([{k:v} for k,v in remaining_clusters.items()]) if isinstance(raw_data, list) else remaining_clusters,
        stats
    )


if __name__ == "__main__":
    main()

In [ ]:
# Add unclustered malware samples
def load_legit_data(json_path):
    with open(json_path, 'r') as f:
        raw_data = json.load(f)
    
    data_type = None
    if isinstance(raw_data, list):
        if all(isinstance(x, list) for x in raw_data):  
            data_type = "list_of_lists"
        elif all(isinstance(x, dict) for x in raw_data):  
            data_type = "list_of_dicts"
    elif isinstance(raw_data, dict) and "embeddings" in raw_data:  
        data_type = "embeddings_dict"
    
    if not data_type:
        raise ValueError("Unrecognized sample format")
    
    return raw_data, data_type

def split_and_save_legit(orig_data, data_type, split_size, output_path_1, output_path_2):
    """Randomly split and save dataset (preserve original format)"""
    # Convert to DataFrame for processing
    if data_type == "list_of_lists":
        df = pd.DataFrame(orig_data)
    elif data_type == "list_of_dicts":
        df = pd.DataFrame([x["embedding"] for x in orig_data])
    elif data_type == "embeddings_dict":
        df = pd.DataFrame(orig_data["embeddings"])
    
    # Random sampling
    if split_size > len(df):
        raise ValueError(f"Split size {split_size} exceeds dataset size {len(df)}")
    
    sampled = df.sample(n=split_size, random_state=42)
    remaining = df.drop(sampled.index)
    
    # Print statistics
    print(f"Split Summary:")
    print(f"├─ Total original samples: {len(df)}")
    print(f"├─ Number of sampled items: {len(sampled)}")
    print(f"└─ Number of remaining items: {len(remaining)}\n")
    
    # Convert back to original format
    def _convert_back(data_df, orig_type):
        if orig_type == "list_of_lists":
            return data_df.values.tolist()
        elif orig_type == "list_of_dicts":
            return [{"embedding": row.tolist()} for _, row in data_df.iterrows()]
        elif orig_type == "embeddings_dict":
            return {"embeddings": data_df.values.tolist()}
    
    def _save_data(data, path, orig_type):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'w') as f:
            json.dump(data, f, indent=2)
    
    _save_data(_convert_back(sampled, data_type), output_path_1, data_type)
    _save_data(_convert_back(remaining, data_type), output_path_2, data_type)

if __name__ == "__main__":
    input_path = "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/mal_embeddings_js_no_cluster.json"          # original unclustered sample path
    output_sampled = "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/nocluster_mal_sample_js.json"     # sampled output path 
    output_remaining = "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/nocluster_mal_remain_js.json"  # remaining output path
    split_num = 100                        

    try:
        orig_data, data_format = load_legit_data(input_path)
        
        split_and_save_legit(
            orig_data = orig_data,
            data_type = data_format,
            split_size = split_num,
            output_path_1 = output_sampled,
            output_path_2 = output_remaining
        )
        
        print(f"Successfully split and saved:\nSampled -> {output_sampled}\nRemaining -> {output_remaining}")
    
    except Exception as e:
        print(f"Processing failed: {str(e)}")

In [ ]:
# Merge Test Set
CONFIG = {
    "legit_test_path": "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/legiti_test_js.json",  # Path to legitimate test samples
    "malware_test_paths": [  # Multiple paths to malware samples (list supported)
        "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/mal_test_js.json",
        "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/nocluster_mal_sample_js.json"
    ]
}

def transform_to_feature_format(df):
    if 'label' not in df.columns:
        raise ValueError("Missing 'label' column in DataFrame")
    
    feature_columns = [col for col in df.columns if col != 'label']
    df[feature_columns] = df[feature_columns].astype(float)
    df['feature'] = df[feature_columns].apply(lambda x: x.tolist(), axis=1)
    
    return df[['feature', 'label']].copy()

def load_labeled_data(json_paths, label):
    """Load data from multiple paths (supports single path or list of paths)"""
    if not isinstance(json_paths, list):
        json_paths = [json_paths]
    
    all_dfs = []
    for path in json_paths:
        with open(path, 'r') as f:
            data = json.load(f)
        
        # Handle dictionary with 'embeddings' key
        if isinstance(data, dict) and "embeddings" in data:
            all_dfs.append(pd.DataFrame(data["embeddings"]))
            continue
        
        # Handle clustered dict structure
        if isinstance(data, dict):
            samples = []
            for cluster in data.values():
                if "samples" in cluster:
                    cluster_samples = cluster["samples"]
                    if cluster_samples and isinstance(cluster_samples[0], dict):
                        samples.extend([s["embedding"] for s in cluster_samples])
                    else:
                        samples.extend(cluster_samples)
            all_dfs.append(pd.DataFrame(samples))
            continue
        
        # Handle flat list structure
        if isinstance(data, list):
            if all(isinstance(x, list) for x in data):
                all_dfs.append(pd.DataFrame(data))
            elif all(isinstance(x, dict) for x in data):
                all_dfs.append(pd.DataFrame([x["embedding"] for x in data]))
    
    if not all_dfs:
        raise ValueError(f"Unrecognized format. Paths: {json_paths}")
    
    return pd.concat(all_dfs).assign(label=label)

def create_test_dataset(legit_path, malware_paths):
    """Create test dataset
    :param legit_path: Path to legitimate samples
    :param malware_paths: List of malware sample paths
    """
    # Load legitimate samples (label 0)
    print(f"[1/3] Loading legitimate samples: {legit_path}")
    legit_df = load_labeled_data(legit_path, 0)
    
    # Load malware samples (label 1)
    print(f"[2/3] Loading malware samples: {len(malware_paths)} files")
    malware_df = load_labeled_data(malware_paths, 1)
    
    # Merge datasets
    print("[3/3] Merging datasets...")
    combined = pd.concat([legit_df, malware_df])
    processed = transform_to_feature_format(combined)
    
    return processed.sample(frac=1, random_state=42).reset_index(drop=True)

# if __name__ == "__main__":
test_set = create_test_dataset(
        CONFIG["legit_test_path"],
        CONFIG["malware_test_paths"]
    )

# Print validation info
print("\n=== Test Set Info ===")
print(f"Total Samples: {len(test_set)}")
print("Label Distribution:")
print(test_set['label'].value_counts())
print("\nFeature Dimension:", len(test_set.iloc[0]['feature']))
print("First 3 Samples:")
print(test_set.head(3))

# Sampling to generate training set

In [ ]:
# Global Sampling


CONFIG = {
    "input_path": "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/nocluster_mal_remain_js.json",  
    "total_samples": 400,  # Number of training samples
    "allow_repeats": True  # Whether to allow repeated sampling
}
# ====================================================================

def main():
    """Main execution function"""
    input_file = CONFIG["input_path"]
    sampling_stats = None  # Explicitly initialize stats variable
    
    try:
        print(f"Reading input file: {input_file}")
        with open(input_file, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)
            
            # Validate format
            if "embeddings" not in raw_data or not isinstance(raw_data["embeddings"], list):
                raise ValueError("Invalid input file format. It must contain a list field 'embeddings'.")
                
    except Exception as e:
        exit(f"Failed to read file: {str(e)}")

    print(f"Starting global random sampling, target size: {CONFIG['total_samples']}")
    try:
        # Convert embeddings to sample format
        all_samples = [{"embedding": emb} for emb in raw_data["embeddings"]]
        
        # Perform sampling
        train_set, sampling_stats = global_random_sampling(
            all_samples,
            CONFIG["total_samples"],
            CONFIG["allow_repeats"]
        )
    except Exception as e:
        exit(f"Error during sampling: {str(e)}")

    # Print statistics if available
    if sampling_stats:
        print_sampling_stats(sampling_stats)
    else:
        print("Warning: No sampling stats generated")

    return train_set

def global_random_sampling(all_samples, total_samples, allow_repeats):
    """Enhanced global random sampling logic"""
    if len(all_samples) == 0:
        raise ValueError("No available samples in input data.")

    # Initialize stats
    sampling_stats = {
        "total_samples": len(all_samples),
        "requested_samples": total_samples,
        "allow_repeats": allow_repeats,
        "actual_sampled": 0,
        "repeats_used": False
    }

    # Handle insufficient sample case
    if not allow_repeats and len(all_samples) < total_samples:
        print(f"Warning: Not enough available samples ({len(all_samples)}), adjusted sample size automatically.")
        total_samples = len(all_samples)
        sampling_stats["requested_samples"] = total_samples

    # Perform sampling
    try:
        selected = random.choices(all_samples, k=total_samples) if allow_repeats else random.sample(all_samples, total_samples)
    except ValueError as e:
        raise ValueError(f"Sampling failed: {str(e)}")

    # Update stats
    sampling_stats["actual_sampled"] = len(selected)
    sampling_stats["repeats_used"] = allow_repeats and (len(selected) > len(all_samples))

    # Extract embedding list
    train_set = [s["embedding"] for s in selected if "embedding" in s]
    return train_set, sampling_stats

def print_sampling_stats(stats):
    """Enhanced stats printing"""
    print("\n=== Sampling Stats ===")
    print(f"Total available samples: {stats['total_samples']}")
    print(f"Requested sample count: {stats['requested_samples']}")
    print(f"Actual samples drawn: {stats['actual_sampled']}")
    print(f"Allow duplicates: {'Yes' if stats['allow_repeats'] else 'No'}")
    print(f"Duplicates used: {'Yes' if stats['repeats_used'] else 'No'}")
    print("=" * 30)

# Run main program
malware_set_1 = main()

In [ ]:
# Cluster-based Sampling

CONFIG = {
    "input_path": "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/mal_train_js.json",  # Input file path
    "total_samples": 400,  # Number of training samples
    "allow_repeats": False  # Allow repeated sampling (when samples are insufficient)
}
# ============================================================

def main():
    """Main execution function"""
    input_file = CONFIG["input_path"]
    
    try:
        print(f"Reading input file: {input_file}")
        with open(input_file, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)
    except Exception as e:
        exit(f"Failed to read file: {str(e)}")

    print(f"Starting sampling, target sample count: {CONFIG['total_samples']}")
    try:
        # Perform cluster-based proportional sampling
        train_set, sampling_stats = cluster_sampling(
            raw_data, 
            CONFIG["total_samples"],
            CONFIG["allow_repeats"]
        )
    except Exception as e:
        exit(f"Sampling error: {str(e)}")

    # Print statistics
    print_sampling_stats(sampling_stats)

    # Return the training set for further use
    return train_set

def print_sampling_stats(stats):
    """Print detailed sampling statistics"""
    print("\n=== Sampling Statistics ===")
    print(f"{'Cluster':<8} {'Allocated':<8} {'Original':<8} {'Sampled':<8} {'Status':<10} {'Note'}")
    print("-" * 60)
    
    for cluster_id, info in stats.items():
        status = "OK" if info["sampled"] == info["allocated"] else "Insufficient"
        note = "With repeats" if info["repeats_used"] else ""
        
        print(f"{cluster_id:<8} {info['allocated']:<8} {info['original_samples']:<8} "
              f"{info['sampled']:<8} {status:<10} {note}")

def cluster_sampling(raw_data, total_samples, allow_repeats):
    """Core sampling logic"""
    # Data preprocessing
    if isinstance(raw_data, list):
        processed_data = {}
        for item in raw_data:
            processed_data.update(item)
    else:
        processed_data = raw_data

    # Validate data structure
    if not isinstance(processed_data, dict):
        raise ValueError("Input data must be a dict or a list of dicts")

    # Build cluster info
    clusters = []
    for cluster_id, details in processed_data.items():
        clusters.append({
            "id": str(cluster_id),
            "size": details.get("size", 0),
            "samples": details.get("samples", [])
        })

    # Perform proportional sampling
    return proportional_sampling(clusters, total_samples, allow_repeats)

def proportional_sampling(clusters, total_samples, allow_repeats):
    """Sampling logic with proportional allocation"""
    # Initialize stats
    sampling_stats = {}
    
    # Compute total size
    total_size = sum(c["size"] for c in clusters)
    if total_size == 0:
        raise ValueError("Total size across all clusters cannot be zero")

    # Allocate samples
    allocations = {}
    remaining = total_samples
    for c in clusters:
        alloc = round(c["size"] / total_size * total_samples)
        allocations[c["id"]] = min(alloc, remaining)
        remaining -= alloc

    # Handle remaining quota
    if remaining > 0:
        largest_cluster = max(clusters, key=lambda x: x["size"])
        allocations[largest_cluster["id"]] += remaining

    # Perform sampling
    train_set = []
    for c in clusters:
        cluster_id = c["id"]
        need = allocations.get(cluster_id, 0)
        samples = c["samples"]
        original_size = len(samples)
        
        # Initialize cluster stats
        stats = {
            "allocated": need,
            "original_samples": original_size,
            "sampled": 0,
            "repeats_used": False
        }

        # Handle insufficient samples
        if original_size < need:
            if not allow_repeats:
                print(f"Warning: Cluster {cluster_id} has insufficient samples (requested {need}, available {original_size})")
                need = original_size
                stats["allocated"] = need
            else:
                print(f"Note: Cluster {cluster_id} allows repeated sampling (requested {need}, available {original_size})")
                stats["repeats_used"] = True

        # Perform sampling
        if need > 0:
            if allow_repeats:
                selected = random.choices(samples, k=need)
            else:
                selected = random.sample(samples, min(need, original_size))
            
            # Record stats
            stats["sampled"] = len(selected)
            sampling_stats[cluster_id] = stats
            
            # Extract embeddings
            train_set.extend([s["embedding"] for s in selected if "embedding" in s])

    return train_set[:total_samples], sampling_stats

# if __name__ == "__main__":
malware_set_2 = main()

In [ ]:
# Merge Training Sets
def load_labeled_data(json_path, label):
    """Universal data loading function (supports multiple formats)"""
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Format 1: list of lists [[0.1,...],...]
    if isinstance(data, list) and all(isinstance(x, list) for x in data):
        return pd.DataFrame(data).assign(label=label)
    
    # Format 2: list of dicts [{"embedding": [...]}, ...]
    if isinstance(data, list) and all("embedding" in x for x in data):
        return pd.DataFrame([x["embedding"] for x in data]).assign(label=label)
    
    # Format 3: dict with "embeddings" key {"embeddings": [[...], ...]}
    if isinstance(data, dict) and "embeddings" in data:
        return pd.DataFrame(data["embeddings"]).assign(label=label)
    
    raise ValueError(f"Unsupported JSON format: {json_path}")

def transform_to_feature_format(df):
    """Convert DataFrame into format with feature list"""
    if 'label' not in df.columns:
        raise ValueError("Missing 'label' column in DataFrame")
    
    feature_columns = [col for col in df.columns if col != 'label']
    df[feature_columns] = df[feature_columns].astype(float)
    df['feature'] = df[feature_columns].apply(lambda x: x.tolist(), axis=1)
    
    return df[['feature', 'label']].copy()

def create_processed_dataset(legit_df, malware_data, samples_per_class):
    """Create dataset ready for machine learning"""
    if isinstance(malware_data, list):
        malware_df = pd.DataFrame(malware_data).assign(label=1)
    elif isinstance(malware_data, dict):
        malware_df = pd.DataFrame(malware_data["embeddings"]).assign(label=1)
    else:
        raise ValueError("Unsupported malware data format")
    
    sampled_legit = legit_df.sample(
        n=samples_per_class,
        replace=len(legit_df) < samples_per_class,
        random_state=42
    )
    sampled_malware = malware_df.sample(
        n=samples_per_class,
        replace=len(malware_df) < samples_per_class,
        random_state=42
    )
    
    combined = pd.concat([sampled_legit, sampled_malware])
    processed_df = transform_to_feature_format(combined)
    final_df = processed_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    return final_df

def main():
    """Main execution function"""
    legit_path = "/Users/zhouxiaoyan/Documents/GIthub/OSS-MPAW/ML/test_train/legiti_train_js.json"  # Legitimate samples path
    samples_per_class = 400  # Number of samples per class

    legit_df = load_labeled_data(legit_path, label=0)

    # Create two training sets
    train_set_1 = create_processed_dataset(legit_df, malware_set_1, samples_per_class)  # Random sampling
    train_set_2 = create_processed_dataset(legit_df, malware_set_2, samples_per_class)  # Cluster-based sampling
    return train_set_1, train_set_2

train_set_1, train_set_2 = main()
print(f"Training set 1 size: {len(train_set_1)}")
print(f"Training set 2 size: {len(train_set_2)}")
print(train_set_1.head())
print(train_set_2.head())

# Train

In [ ]:
endan = []
endan2 = []

## Traditional machine

In [ ]:
def preprocess_data(X_train, X_test):
    scaler = StandardScaler()

    X_train_expanded = pd.DataFrame(X_train['feature'].tolist())
    X_train_expanded.fillna(0, inplace=True)  
    X_train_scaled = scaler.fit_transform(X_train_expanded)  

    X_test_expanded = pd.DataFrame(X_test['feature'].tolist())
    X_test_expanded.fillna(0, inplace=True)  
    X_test_scaled = scaler.transform(X_test_expanded)  

    return X_train_scaled, X_test_scaled, X_train['label'], X_test['label']

def train_and_predict(model, X_train, X_test, y_train, param_grid):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    X_test_expanded = X_test.astype(float)
    y_pred = best_model.predict(X_test_expanded)

    return y_pred, best_model, grid_search.best_params_, grid_search.best_score_

def traditional_machine(train_set_1, test_set):
    #  use train_set_1 and test_set
    X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(train_set_1, test_set)

    models_and_params = {
        # LogisticRegression(): {'C': [0.001, 0.01, 0.1, 1, 10, 100]},
        # DecisionTreeClassifier(): {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]},
        # RandomForestClassifier(): {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]},
        # SVC(): {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
        # KNeighborsClassifier(): {'n_neighbors': [3, 5, 7]},
        # MLPClassifier(): {'hidden_layer_sizes': [(50,), (100,), (50, 50)]}
        LogisticRegression(): {'C': [0.1]},
        DecisionTreeClassifier(): {'max_depth': [30], 'min_samples_split': [5]},
        RandomForestClassifier(): {'n_estimators': [200], 'max_depth': [None], 'min_samples_split': [5]},
        KNeighborsClassifier(): {'n_neighbors': [5]},
        MLPClassifier(): {'hidden_layer_sizes': [(50,50)]}
    }

    for model, params in models_and_params.items():
        model_name = model.__class__.__name__
        print(f"Tuning {model_name} parameters and performing cross-validation evaluation...")

        y_pred, best_model, best_params, best_score = train_and_predict(model, X_train_scaled, X_test_scaled, y_train, params)

        # Output best model and parameters
        print(f"Best model: {best_model}")
        print(f"Best parameters: {best_params}")
        print(f"Best cross-validation accuracy: {best_score:.2f}")

        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        print(f"{model_name} test accuracy: {accuracy:.10f}")
        print(f"{model_name} Precision: {precision:.10f}")
        print(f"{model_name} Recall: {recall:.10f}")
        print(f"{model_name} F1-Score: {f1:.10f}")
        endan.append([model_name, best_params, accuracy, precision, recall, f1])
        cm = confusion_matrix(y_test, y_pred)
        print(f"{model_name} Confusion Matrix:\n{cm}")

# Baseline traditional machine train
traditional_machine(train_set_1, test_set)

In [ ]:
def preprocess_data(X_train, X_test):
    scaler = StandardScaler()

    X_train_expanded = pd.DataFrame(X_train['feature'].tolist())
    X_train_expanded.fillna(0, inplace=True)  
    X_train_scaled = scaler.fit_transform(X_train_expanded)  

    X_test_expanded = pd.DataFrame(X_test['feature'].tolist())
    X_test_expanded.fillna(0, inplace=True)  
    X_test_scaled = scaler.transform(X_test_expanded)  

    return X_train_scaled, X_test_scaled, X_train['label'], X_test['label']

def train_and_predict(model, X_train, X_test, y_train, param_grid):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    X_test_expanded = X_test.astype(float)
    y_pred = best_model.predict(X_test_expanded)

    return y_pred, best_model, grid_search.best_params_, grid_search.best_score_

def traditional_machine(train_set_2, test_set):
    # use train_set_2 和 test_set
    X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(train_set_2, test_set)

    models_and_params = {
        # LogisticRegression(): {'C': [0.001, 0.01, 0.1, 1, 10, 100]},
        # DecisionTreeClassifier(): {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]},
        # RandomForestClassifier(): {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]},
        # KNeighborsClassifier(): {'n_neighbors': [3, 5, 7]},
        # MLPClassifier(): {'hidden_layer_sizes': [(50,), (100,), (50, 50)]}
        LogisticRegression(): {'C': [0.1]},
        DecisionTreeClassifier(): {'max_depth': [30], 'min_samples_split': [5]},
        RandomForestClassifier(): {'n_estimators': [200], 'max_depth': [None], 'min_samples_split': [5]},
        KNeighborsClassifier(): {'n_neighbors': [5]},
        MLPClassifier(): {'hidden_layer_sizes': [(50,50)]}

    }

    for model, params in models_and_params.items():
        model_name = model.__class__.__name__
        print(f"Tuning {model_name} parameters and performing cross-validation evaluation...")

        y_pred, best_model, best_params, best_score = train_and_predict(model, X_train_scaled, X_test_scaled, y_train, params)

        # Output best model and parameters
        print(f"Best model: {best_model}")
        print(f"Best parameters: {best_params}")
        print(f"Best cross-validation accuracy: {best_score:.2f}")

        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        print(f"{model_name} test accuracy: {accuracy:.10f}")
        print(f"{model_name} Precision: {precision:.10f}")
        print(f"{model_name} Recall: {recall:.10f}")
        print(f"{model_name} F1-Score: {f1:.10f}")
        endan2.append([model_name, best_params, accuracy, precision, recall, f1])
        cm = confusion_matrix(y_test, y_pred)
        print(f"{model_name} Confusion Matrix:\n{cm}")


#  cluster machine train
traditional_machine(train_set_2, test_set)

## CNN

In [ ]:
def preprocess_data(train_set, test_set):
    # Preprocess the training and testing datasets
    scaler = StandardScaler()
    X_train = pd.DataFrame(train_set['feature'].tolist()).astype(float)
    X_train.fillna(0, inplace=True)  # Fill NaN values
    X_train_scaled = scaler.fit_transform(X_train)  # Standardize training set

    # Extract and preprocess test set features
    X_test = pd.DataFrame(test_set['feature'].tolist()).astype(float)
    X_test.fillna(0, inplace=True)  # Fill NaN values
    X_test_scaled = scaler.transform(X_test)  # Standardize test set

    # Get labels
    y_train = train_set['label'].values
    y_test = test_set['label'].values

    return X_train_scaled, X_test_scaled, y_train, y_test

def train_cnn(train_set, test_set):
    # Data preprocessing
    X_train, X_test, y_train, y_test = preprocess_data(train_set, test_set)

    # Reshape data for CNN
    X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Convert y labels to categorical format (for binary classification)
    y_train_categorical = to_categorical(y_train)
    y_test_categorical = to_categorical(y_test)

    # CNN hyperparameter grid search
    param_grid = {
        'epochs': [20],
        'batch_size': [32],
        'filters': [32],
        'kernel_size': [5]
    }

    best_params = None
    best_performance = 0.0
    best_precision = 0
    best_recall = 0
    best_f1 = 0

    # Hyperparameter tuning via grid search
    for params in ParameterGrid(param_grid):
        print("\nCurrent Parameters:", params)

        # Build CNN model
        cnn_model = Sequential()
        cnn_model.add(Conv1D(filters=params['filters'], kernel_size=params['kernel_size'], activation='relu', input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2])))
        cnn_model.add(MaxPooling1D(pool_size=2))
        cnn_model.add(Flatten())
        cnn_model.add(Dense(1, activation='sigmoid'))  # Binary classification using sigmoid activation

        cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the model
        cnn_model.fit(X_train_cnn, y_train, epochs=params['epochs'], batch_size=params['batch_size'], validation_split=0.1)

        # Predict on test set
        cnn_predictions = cnn_model.predict(X_test_cnn)
        cnn_predictions = (cnn_predictions > 0.5).astype(int)  # Convert to binary labels

        # Evaluate performance
        cnn_accuracy = accuracy_score(y_test, cnn_predictions)
        cnn_precision = precision_score(y_test, cnn_predictions)
        cnn_recall = recall_score(y_test, cnn_predictions)
        cnn_f1 = f1_score(y_test, cnn_predictions)

        print("CNN Accuracy:", cnn_accuracy)
        print("CNN Precision:", cnn_precision)
        print("CNN Recall:", cnn_recall)
        print("CNN F1-Score:", cnn_f1)

        cnn_cm = confusion_matrix(y_test, cnn_predictions)
        print("CNN Confusion Matrix:\n", cnn_cm)

        # Update best performance
        if cnn_accuracy > best_performance:
            best_performance = cnn_accuracy
            best_params = params
            best_precision = cnn_precision
            best_recall = cnn_recall
            best_f1 = cnn_f1

    # Output best results
    print("\n=== Best CNN Model Results ===")
    print("Best Parameters:", best_params)
    print("Best Performance (Accuracy):", best_performance)
    print("CNN Recall:", best_recall)
    print("CNN Precision:", best_precision)
    print("CNN F1-Score:", best_f1)

    # Record best result
    endan.append(['CNN', best_params, best_performance, best_precision, best_recall, best_f1])

# baseline
train_cnn(train_set_1, test_set)

In [ ]:
def preprocess_data(train_set, test_set):
    scaler = StandardScaler()

    # Extract and preprocess training set features
    X_train = pd.DataFrame(train_set['feature'].tolist()).astype(float)
    X_train.fillna(0, inplace=True)  # Fill NaN values
    X_train_scaled = scaler.fit_transform(X_train)  # Standardize training set

    # Extract and preprocess test set features
    X_test = pd.DataFrame(test_set['feature'].tolist()).astype(float)
    X_test.fillna(0, inplace=True)  # Fill NaN values
    X_test_scaled = scaler.transform(X_test)  # Standardize test set

    # Get labels
    y_train = train_set['label'].values
    y_test = test_set['label'].values

    return X_train_scaled, X_test_scaled, y_train, y_test

def train_cnn(train_set, test_set):
    # Data preprocessing
    X_train, X_test, y_train, y_test = preprocess_data(train_set, test_set)

    # Reshape data for CNN
    X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Convert y labels to categorical format (if binary classification)
    y_train_categorical = to_categorical(y_train)
    y_test_categorical = to_categorical(y_test)

    # CNN hyperparameter grid search
    param_grid = {
        'epochs': [20],
        'batch_size': [32],
        'filters': [32],
        'kernel_size': [5]
    }

    best_params = None
    best_performance = 0.0
    best_precision = 0
    best_recall = 0
    best_f1 = 0

    # Perform hyperparameter tuning using grid search
    for params in ParameterGrid(param_grid):
        print("\nCurrent Parameters:", params)

        # Build CNN model
        cnn_model = Sequential()
        cnn_model.add(Conv1D(filters=params['filters'], kernel_size=params['kernel_size'], activation='relu', input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2])))
        cnn_model.add(MaxPooling1D(pool_size=2))
        cnn_model.add(Flatten())
        cnn_model.add(Dense(1, activation='sigmoid'))  # Binary classification with sigmoid activation

        cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the model
        cnn_model.fit(X_train_cnn, y_train, epochs=params['epochs'], batch_size=params['batch_size'], validation_split=0.1)

        # Predict on test set
        cnn_predictions = cnn_model.predict(X_test_cnn)
        cnn_predictions = (cnn_predictions > 0.5).astype(int)  # Convert to binary labels

        # Calculate performance metrics
        cnn_accuracy = accuracy_score(y_test, cnn_predictions)
        cnn_precision = precision_score(y_test, cnn_predictions)
        cnn_recall = recall_score(y_test, cnn_predictions)
        cnn_f1 = f1_score(y_test, cnn_predictions)

        print("CNN Accuracy:", cnn_accuracy)
        print("CNN Precision:", cnn_precision)
        print("CNN Recall:", cnn_recall)
        print("CNN F1-Score:", cnn_f1)

        cnn_cm = confusion_matrix(y_test, cnn_predictions)
        print("CNN Confusion Matrix:\n", cnn_cm)

        # Update best performance
        if cnn_accuracy > best_performance:
            best_performance = cnn_accuracy
            best_params = params
            best_precision = cnn_precision
            best_recall = cnn_recall
            best_f1 = cnn_f1

    # Output best parameters and results
    print("\n=== Best CNN Model Results ===")
    print("Best Parameters:", best_params)
    print("Best Performance (Accuracy):", best_performance)
    print("CNN Recall:", best_recall)
    print("CNN Precision:", best_precision)
    print("CNN F1-Score:", best_f1)

    # Record best result
    endan2.append(['CNN', best_params, best_performance, best_precision, best_recall, best_f1])

# Cluster-based training
train_cnn(train_set_2, test_set)

## Print

In [ ]:
from tabulate import tabulate

def merge_results(baseline, cluster):
    merged = []
    headers = [
        "Model",
        "Baseline Acc", "Cluster Acc",
        "Baseline Prec", "Cluster Prec",
        "Baseline Recall", "Cluster Recall",
        "Baseline F1", "Cluster F1"
    ]
    
    for b, c in zip(baseline, cluster):
        merged.append([
            b[0],
            f"{b[2]:.4f}", f"{c[2]:.4f}",
            f"{b[3]:.4f}", f"{c[3]:.4f}",
            f"{b[4]:.4f}", f"{c[4]:.4f}",
            f"{b[5]:.4f}", f"{c[5]:.4f}"
        ])
    return tabulate(merged, headers=headers, tablefmt="fancy_grid")

print(merge_results(endan, endan2))